# Model 1

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [ ]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_1b = pd.read_csv("data/informatikkurse.csv")
data_model_1b = data_model_1b[data_model_1b['model_1_b'] == 1].copy()
data_model_1b

c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   Unnamed: 0  veranstaltung_id  \
0           1           1717848   
1           2           1717914   
2           3           1717915   
3           4           1717989   
4           5           1717990   

                                 veranstaltung_titel kursbeschreibung  \
0    Übung zu: Didaktische Grundlagen für Informatik              NaN   
1  Seminar zu: Lehren und Lernen im Fach Informat...              NaN   
2                  Übung zu: Ethik in der Informatik              NaN   
3                  Übung zu: Ethik in der Informatik              NaN   
4                  infSP2F-01a: Softwareprojekt (2F)              NaN   

  kursformat_original sprache_original lernziele lernmethode literatur  \
0               Übung              NaN       NaN         NaN       NaN   
1             Seminar              NaN       NaN         NaN       NaN   
2               Übung              NaN       NaN         NaN       NaN   
3               Übung              NaN       NaN      

C:\Users\mhu\AppData\Local\Temp\ipykernel_12564\3578484799.py:25: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_1b = pd.read_csv("data/informatikkurse.csv")


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [2]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [3]:
# Anzeige der Spaltennamen von data_model_1b
print(data_model_1b.columns.tolist())

data_model_1b = data_model_1b[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_1b.head()

data_model_1b = data_model_1b.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_1b['titel_kursbesch'] = data_model_1b['veranstaltung_titel'] + ' ' + data_model_1b['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_1b['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['Unnamed: 0', 'veranstaltung_id', 'veranstaltung_titel', 'kursbeschreibung', 'kursformat_original', 'sprache_original', 'lernziele', 'lernmethode', 'literatur', 'voraussetzungen', 'zusatzinformationen', 'anmerkungen', 'pruefung', 'dozierende', 'teilnehmerzahl', 'sws', 'ects', 'url', 'nummer', 'pfad', 'scrape_datum', 'hochschule_id', 'hochschule_name', 'fakultaet_id', 'fakultaet_name', 'organisation_id', 'organisation_name', 'semester_id', 'semester_name', 'sprache_id', 'sprache_name', 'kursformat_id', 'kursformat_name', 'matchingart_id', 'matchingart_name', 'lehr_und_forschungsbereich_id', 'bio_info', 'ingen_info', 'medien_info', 'medizin_info', 'wirt_info', 'recht_info', 'geo_info', 'bau_info', 'base_info', 'model_1_b', 'model_2', 'model_3', 'model_4']


## Stopwords

In [4]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

# Model

Wir definieren unser Model.

In [5]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [ ]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
topic_model_1b = BERTopic(
  embedding_model=embedding_model,
  #min_topic_size=10,
  nr_topics=20, 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 15,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [ ]:
topic_model_1b.fit(docs)
topic_model_1b.get_topic_info()

2026-02-05 14:46:21,775 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1194/1194 [00:25<00:00, 46.23it/s] 
2026-02-05 14:46:48,390 - BERTopic - Embedding - Completed ✓
2026-02-05 14:46:48,391 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-05 14:47:11,699 - BERTopic - Dimensionality - Completed ✓
2026-02-05 14:47:11,701 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-05 14:47:15,677 - BERTopic - Cluster - Completed ✓
2026-02-05 14:47:15,677 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-02-05 14:47:18,245 - BERTopic - Representation - Completed ✓
2026-02-05 14:47:18,251 - BERTopic - Topic reduction - Reducing number of topics
2026-02-05 14:47:18,281 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-05 14:47:21,336 - BERTopic - Representation - Completed ✓
2026-02-05 14:47:21,345 - BERTopic - Topic reduction -

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7462,-1_data_systems_informatik_software,"[data, systems, informatik, software, systeme,...",[Cloud Computing (IN2073) 1. Cloud Archite...
1,0,22356,0_informatik_data_software_systems,"[informatik, data, software, systems, algorith...",[Einführung in die Technische Informatik Die E...
2,1,2044,1_bioinformatik_biometrie_medizinische informa...,"[bioinformatik, biometrie, medizinische inform...","[Bioinformatik , Einführung in die Bioinforma..."
3,2,1749,2_machine learning_robotics_deep learning_robotik,"[machine learning, robotics, deep learning, ro...",[Dodo Alive! - Resurrecting the Dodo with Robo...
4,3,1242,3_security_privacy_secure_cryptography,"[security, privacy, secure, cryptography, secu...",[Introduction to Modern Cryptography IMPORTANT...
5,4,899,4_process mining_operations research_managemen...,"[process mining, operations research, manageme...",[Master-Seminar - Digital Transformation (IN21...
6,5,734,5_forschungsprojekt_führt forschungsprojekt_wi...,"[forschungsprojekt, führt forschungsprojekt, w...",[Inf-WissArb: Wissenschaftliches Arbeiten Lern...
7,6,604,6_cloud_web science_social media web_cloud com...,"[cloud, web science, social media web, cloud c...",[WInf-SMWS: Social Media und Web Science Web S...
8,7,384,7_kommunikationswissenschaft_learning_sprach k...,"[kommunikationswissenschaft, learning, sprach ...",[Praktikum DBMS: Deep Learning for TextImaging...
9,8,284,8_soft skills_soft_skills_skills technische ko...,"[soft skills, soft, skills, skills technische ...",[Übung: 2.01.085-PB-2 Soft Skills und Technisc...


## Outlier Reduzieren, Topics mergen

In [ ]:
topic_model_1b.visualize_topics()
topic_model_1b.visualize_hierarchy() # Hilft extrem zu sehen, welche Themen zusammengehören

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# Outlier reduzieren
# BERTopic auf Test-Daten anwenden
# topics, probs = topic_model_1b.transform(docs)
# print(topic_model_1b.get_topic_freq())

# # Outlier reduzieren
# topics = topic_model_1b.reduce_outliers(docs, topics)

In [ ]:
topic_model_1b.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7462,-1_data_systems_informatik_software,"[data, systems, informatik, software, systeme,...",[Cloud Computing (IN2073) 1. Cloud Archite...
1,0,22356,0_informatik_data_software_systems,"[informatik, data, software, systems, algorith...",[Einführung in die Technische Informatik Die E...
2,1,2044,1_bioinformatik_biometrie_medizinische informa...,"[bioinformatik, biometrie, medizinische inform...","[Bioinformatik , Einführung in die Bioinforma..."
3,2,1749,2_machine learning_robotics_deep learning_robotik,"[machine learning, robotics, deep learning, ro...",[Dodo Alive! - Resurrecting the Dodo with Robo...
4,3,1242,3_security_privacy_secure_cryptography,"[security, privacy, secure, cryptography, secu...",[Introduction to Modern Cryptography IMPORTANT...
5,4,899,4_process mining_operations research_managemen...,"[process mining, operations research, manageme...",[Master-Seminar - Digital Transformation (IN21...
6,5,734,5_forschungsprojekt_führt forschungsprojekt_wi...,"[forschungsprojekt, führt forschungsprojekt, w...",[Inf-WissArb: Wissenschaftliches Arbeiten Lern...
7,6,604,6_cloud_web science_social media web_cloud com...,"[cloud, web science, social media web, cloud c...",[WInf-SMWS: Social Media und Web Science Web S...
8,7,384,7_kommunikationswissenschaft_learning_sprach k...,"[kommunikationswissenschaft, learning, sprach ...",[Praktikum DBMS: Deep Learning for TextImaging...
9,8,284,8_soft skills_soft_skills_skills technische ko...,"[soft skills, soft, skills, skills technische ...",[Übung: 2.01.085-PB-2 Soft Skills und Technisc...


## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [ ]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
topic_model_1b.update_topics(docs, representation_model=rep)
topic_model_1b.get_topic_info() 

100%|██████████| 20/20 [03:36<00:00, 10.83s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,7462,-1_Computer Graphics and AI,[Computer Graphics and AI],[Cloud Computing (IN2073) 1. Cloud Archite...
1,0,22356,0_Computer Science Education,[Computer Science Education],[Einführung in die Technische Informatik Die E...
2,1,2044,1_Introduction to Bioinformatics,[Introduction to Bioinformatics],"[Bioinformatik , Einführung in die Bioinforma..."
3,2,1749,2_Robotics and AI,[Robotics and AI],[Dodo Alive! - Resurrecting the Dodo with Robo...
4,3,1242,3_Security and Privacy,[Security and Privacy],[Introduction to Modern Cryptography IMPORTANT...
5,4,899,4_Process Mining Fundamentals,[Process Mining Fundamentals],[Master-Seminar - Digital Transformation (IN21...
6,5,734,5_Scientific Research Methods,[Scientific Research Methods],[Inf-WissArb: Wissenschaftliches Arbeiten Lern...
7,6,604,6_Web Science and Social Media,[Web Science and Social Media],[WInf-SMWS: Social Media und Web Science Web S...
8,7,384,7_Deep Learning for Text Analysis,[Deep Learning for Text Analysis],[Praktikum DBMS: Deep Learning for TextImaging...
9,8,284,8_Soft Skills Training,[Soft Skills Training],[Übung: 2.01.085-PB-2 Soft Skills und Technisc...


In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")